In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the Data

In [2]:
train = pd.read_csv('data/train.csv')
train_labels = pd.read_csv('data/train_labels.csv')
test = pd.read_csv('data/test.csv')
specs = pd.read_csv('data/specs.csv')
sample = pd.read_csv('data/sample_submission.csv')

Source: https://www.kaggle.com/erikbruin/data-science-bowl-2019-eda-and-baseline

So we have 11 million rows and just 11 columns. However, Kaggle provided the following note: Note that the training set contains many installation_ids which never took assessments, whereas every installation_id in the test set made an attempt on at least one assessment.

As there is no point in keeping training data that cannot be used for training anyway, I am getting rid of the installation_ids that never took an assessment

In [3]:
# filtering by ids that took assessments
ids_w_assessments = train[train['type'] == 'Assessment']['installation_id'].drop_duplicates()
train = train[train['installation_id'].isin(ids_w_assessments)]

In [10]:
# convert timestamp to correct datetime type
train['timestamp'] = pd.to_datetime(train['timestamp'], infer_datetime_format=True)

In [24]:
from tqdm.notebook import tqdm

In [25]:
def process_data(df):
    types = train['type'].unique() + '_counts'
    totals = ['total_game_time', 'total_event_count']
    flattened = pd.DataFrame(columns=list(train.columns) + list(types) + totals)
    
    for installation in tqdm(df['installation_id'].unique()):
        for world in df[df['installation_id'] == installation]['world'].unique():
            sl = df[df['installation_id'] == installation].copy()
            sl = sl[sl['world'] == world]
            
            condensed = pd.DataFrame(columns=sl.columns)

            for key, group in sl.groupby(['game_session']):
                temp = pd.DataFrame(index=range(1,2), columns=sl.columns)

                unique_cols = ['event_id', 'game_session', 'installation_id', 'title', 'type', 'world']
                for col in unique_cols:
                    temp[col] = group[col].unique()[0]

                last_val_cols = ['timestamp', 'event_count', 'game_time']
                for col in last_val_cols:
                    temp[col] = group[col].iloc[-1]

                condensed = condensed.append(temp, ignore_index=True)
                
                # group activities by assessment
                condensed['assessment_group'] = np.nan
                list_assessement_groups = np.arange(len(condensed.loc[condensed['type'] == 'Assessment', 'assessment_group']))
                condensed.loc[condensed['type'] == 'Assessment', 'assessment_group'] = list_assessement_groups
                condensed['assessment_group'].fillna(method='backfill', inplace=True)
                
                cm = condensed.merge(train_labels, on=['game_session', 'installation_id', 'title'], how='left')
                
                for key, group in cm.groupby(['assessment_group']):
                    temp1 = pd.DataFrame(index=range(1,2), columns=list(train.columns) + list(types) + totals)
                    temp_counts = group['type'].value_counts()

                    for col in group.columns:
                        temp1[col] = group[col].iloc[-1]

                    for key, value in temp_counts.items():
                        temp1[key + '_counts'] = value

                    temp1['total_game_time'] = sum(group['game_time'])
                    temp1['total_event_count'] = sum(group['event_count'])

                    flattened = flattened.append(temp1, ignore_index=True)
                    
    return flattened

In [26]:
flattened = process_data(train)

KeyboardInterrupt: 

In [ ]:
flattened.shape

In [ ]:
flattened.to_pickle('data/ftrain.pkl')